In [2]:
import sys
sys.executable

'C:\\Users\\DELL\\miniconda3\\envs\\min_ds-env\\python.exe'

In [3]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
import os
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không
from selenium import webdriver

In [6]:
#PATH = 'C:\Program Files (x86)\chromedriver.exe'

def GetUsers():
    Users = []
    
    url = 'https://soundcloud.com/discover/sets/charts-top:all-music:vn'
    #url = 'https://soundcloud.com/discover/sets/charts-top:hiphoprap:vn'
    main_url = 'https://soundcloud.com'
    
    browser_1 = webdriver.Chrome(executable_path="chromedriver.exe")
    browser_1.get(url)
    time.sleep(1)
    
    # To make it load more scroll to the bottom of the page (repeat if you want to)
    
    lenOfPage_1 = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match_1 = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
                match = True
                    
    html_source = browser.page_source
    browser.quit()
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('a', {'class' : 'trackItem__username sc-link-light sc-link-secondary'}):
        UsersTemp = []
        
        UsersTemp.append(url)
        UsersTemp.append(id.get('href')) #lay link de di den tracks va playlist cua moi user
        UsersTemp.append(id.text) #lay username
        
        browser = webdriver.Chrome(executable_path="chromedriver.exe")
        browser.get(url)
        time.sleep(1)
    
    # To make it load more scroll to the bottom of the page (repeat if you want to)
    
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match = False
        while(match == False):
            lastCount = lenOfPage
            time.sleep(1)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage:
                    match = True

        html_source = browser.page_source
        browser.quit()
        
        Users.append(UsersTemp)
        
    return Users

def GetTracks(Users):
    TracksOfAllUsers = []
    
    for i in range(len(Users)):
        TrackOfOneUser = []
        
        url = 'https://soundcloud.com'
        url = url + str(Users[i][1]) + '/tracks'
    
        browser = webdriver.Chrome(executable_path="chromedriver.exe")
        browser.get(url)
        time.sleep(3)

        # To make it load more scroll to the bottom of the page (repeat if you want to)
        
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match = False
        while(match == False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage:
                match = True
                
        html_source = browser.page_source
        browser.quit()
        
        soup = BeautifulSoup(html_source, 'html.parser')
        check = soup.findAll('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        if check == []:
            TrackOfOneUser.append(Users[i][2])
            TrackOfOneUser.append('None')
            TracksOfAllUsers.append(TrackOfOneUser)
            print('OK')
        else:
            for id in check:
                TrackOfOneUser = []
                TrackOfOneUser.append(Users[i][2])
                TrackOfOneUser.append(id.text) #lay track cua 1 user
                TracksOfAllUsers.append(TrackOfOneUser)
    return TracksOfAllUsers

def GetPlaylist(Users):
    PlaylistOfAllUsers = []
    
    for i in range(len(Users)):
        InfoOfOnePlaylist = []
        
        url = 'https://soundcloud.com'
        url = url + str(Users[i][1]) + '/sets' #lay duong dan den playlist
        
        browser = webdriver.Chrome(executable_path="chromedriver.exe")
        browser.get(url)
        time.sleep(3)

        # To make it load more scroll to the bottom of the page (repeat if you want to)
        
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match = False
        while(match == False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage:
                match = True
                
        html_source = browser.page_source
        browser.quit()
        
        soup = BeautifulSoup(html_source, 'html.parser')
        check = soup.findAll('div', {'class' : 'sound__content'})
        if check == []:
            InfoOfOnePlaylist.append(Users[i][2])
            InfoOfOnePlaylist.append('None')
            InfoOfOnePlaylist.append('None')
            PlaylistOfAllUsers.append(InfoOfOnePlaylist)
        else:        
            for id in check:
                Title = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
                Tracks = id.findAll('span', {'class' : 'compactTrackListItem__trackTitle sc-text-primary sc-text-h4'})
                for Track in Tracks:
                    InfoOfOnePlaylist = []
                    InfoOfOnePlaylist.append(Users[i][2])
                    InfoOfOnePlaylist.append(Title.text)
                    InfoOfOnePlaylist.append(Track.text)
                    PlaylistOfAllUsers.append(InfoOfOnePlaylist)
    return PlaylistOfAllUsers

def writeUsersToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['Url', 'Users Link', 'User Name'])
    df.to_csv(filename, index = True, sep = '\t')
    
def writeTracksToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['User Name', 'Tracks Of One User'])
    df.to_csv(filename, index = True, sep = '\t')

def writePlaylistsToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['User Name', 'Playlists Of One User', 'Tracks Of One Playlist'])
    df.to_csv(filename, index = True, sep = '\t')

In [7]:
Users = GetUsers()
#print(Users[0])
Tracks = GetTracks(Users)
#Playlists = GetPlaylist(Users)
#print(Tracks)
#writeUsersToCsv(Users, './Crawl_data/users.csv')
#writeTracksToCsv(Tracks, './Crawl_data/tracks.csv')
#writePlaylistsToCsv(Playlists, './Crawl_data/playlists.csv')
#print(len(Users[0]))
#print(len(Users[1]))
#print(len(Users[2]))
#GetTracks(Users)
#print(len(GetPlaylist(Users)))

OK


WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.69)


In [37]:
def TestGetTracks(Users):
    TracksOfAllUsers = []
    
    url = 'https://soundcloud.com/winnodamidside'
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    browser.get(url)
    time.sleep(3)

    # To make it load more scroll to the bottom of the page (repeat if you want to)
    
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
            match = True
            
    html_source = browser.page_source
    browser.quit()

    soup = BeautifulSoup(html_source, 'html.parser')
    TrackOfOneUser = []
    check = soup.findAll('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
    if check == []:
        TrackOfOneUser.append(Users[0][2])
        TrackOfOneUser.append('None')
        TracksOfAllUsers.append(TrackOfOneUser)
    else:
        for id in check:
            TrackOfOneUser = []
            TrackOfOneUser.append(Users[0][2])
            TrackOfOneUser.append(id.text)
            TracksOfAllUsers.append(TrackOfOneUser)
    return TracksOfAllUsers

def TestGetPlaylist(Users):
    url = 'https://soundcloud.com/nguy-n-to-n-401524044/sets'
    
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    browser.get(url)
    time.sleep(3)

    # To make it load more scroll to the bottom of the page (repeat if you want to)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    html_source = browser.page_source
    browser.quit()
        
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('div', {'class' : 'sound__content'}):
        Title = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        print(Title.text)
        Tracks = id.findAll('span', {'class' : 'compactTrackListItem__trackTitle sc-text-primary sc-text-h4'})
        for Track in Tracks:
            print(Track.text)
        print('\n')

In [38]:
tr = TestGetTracks(Users)
writeTracksToCsv(tr, './Crawl_data/tracks.csv')
#TestGetPlaylist(Users)